# Example: CitiBike data

Adapted from Kelsey Jordahl
https://gist.github.com/kjordahl/5957573

In [1]:
import requests  
import json
from pyproj import Proj
from shapely.geometry import Point
import geopandas as gp

NYC borough boundaries downloaded from [Bytes of the Big Apple](http://www.nyc.gov/html/dcp/download/bytes/nybb_13a.zip)

In [2]:
boros = gp.GeoDataFrame.from_file('nybb_15b/nybb.shp')
boros

,BoroCode,BoroName,Shape_Leng,Shape_Area,geometry
0,5,Staten Island,330466.075042,1.623827e+09,(POLYGON ((970217.0223999023 145643.3322143555...
1,3,Brooklyn,741185.900596,1.937597e+09,(POLYGON ((1021176.479003906 151374.7969970703...
2,4,Queens,897040.298576,3.045168e+09,(POLYGON ((1029606.076599121 156073.8142089844...
3,1,Manhattan,358408.460709,6.364467e+08,(POLYGON ((981219.0557861328 188655.3157958984...
4,2,Bronx,464400.198868,1.186973e+09,(POLYGON ((1012821.805786133 229228.2645874023...


Load real time bike station data from [CitiBike](http://citibikenyc.com) json API:

In [3]:
# get the bike data and convert it to a ditcionary
endpoint_url = 'http://citibikenyc.com/stations/json'
response = requests.get(endpoint_url)
data = json.loads(response.text)


In [4]:
# convert the relevant part to a geodataframe
df = gp.GeoDataFrame(data['stationBeanList'])
df.head()

,altitude,availableBikes,availableDocks,city,id,landMark,lastCommunicationTime,latitude,location,longitude,postalCode,stAddress1,stAddress2,stationName,statusKey,statusValue,testStation,totalDocks
0,,2,39,,520,,2019-02-19 02:26:50 AM,40.759923,,-73.976485,,W 52 St & 5 Ave,,W 52 St & 5 Ave,1,In Service,False,41
1,,9,45,,72,,2019-02-19 02:25:35 AM,40.767272,,-73.993929,,W 52 St & 11 Ave,,W 52 St & 11 Ave,1,In Service,False,55
2,,16,15,,79,,2019-02-19 02:25:51 AM,40.719116,,-74.006667,,Franklin St & W Broadway,,Franklin St & W Broadway,1,In Service,False,33
3,,26,1,,82,,2019-02-19 02:28:34 AM,40.711174,,-74.000165,,St James Pl & Pearl St,,St James Pl & Pearl St,1,In Service,False,27
4,,38,24,,83,,2019-02-19 02:28:30 AM,40.683826,,-73.976323,,Atlantic Ave & Fort Greene Pl,,Atlantic Ave & Fort Greene Pl,1,In Service,False,62


In [5]:
# there is one row for each bike station.  How many stations are there? 
len(df)

814

In [6]:
# in the file above, there are lon-lats, but no geometry field
# we need to set that up

s = gp.GeoSeries([Point(x, y) for x, y in zip(df['longitude'], df['latitude'])])
df['geometry'] = s
df.crs = {'init': 'epsg:4326', 'no_defs': True}
df.geometry.total_bounds

array([-74.0836394 ,  40.65539977, -73.9077436 ,  40.81439444])

In [7]:
# make sure they are on the same CRS.  
# checking the bounds is a nice way of seeing this
df.to_crs(boros.crs, inplace=True)
df.geometry.total_bounds

array([ 961064.73659426,  178056.37414829, 1009803.53178432,
        235986.42166716])

In [8]:
# the geometry objects can do lots of cool stuff.  For example: 

manhattan = boros.geometry[3]
in_mn = df.geometry.within(manhattan)
print(sum(in_mn), 'stations in Manhattan')

414 stations in Manhattan


# Your turn

You can read about the range of operations available in geopandas here: 
    
http://geopandas.org/index.html

Your assignment is to: 

1. Calculate how many stations are in each borough
2. Calculate how many bikes are currently available in each borough
3. Read about and try at least two new spatial or geometric operations (beyond what I've covered here). 

In [34]:
bronx = boros.geometry[4]
in_bx = df.geometry.within(bronx)

In [35]:
queens = boros.geometry[2]
in_Q = df.geometry.within(queens)

In [36]:
brooklyn = boros.geometry[1]
in_bk = df.geometry.within(brooklyn)

In [37]:
statenisland = boros.geometry[0]
in_si = df.geometry.within(statenisland)

In [38]:
print(sum(in_mn), 'stations in Manhattan')
print(sum(in_bx), 'stations in Bronx')
print(sum(in_Q), 'stations in Queens')
print(sum(in_bk), 'stations in Brooklyn')
print(sum(in_si), 'stations in Staten Island')

414 stations in Manhattan
0 stations in Bronx
82 stations in Queens
268 stations in Brooklyn
0 stations in Staten Island


In [16]:
df.availableBikes

0       2
1       9
2      16
3      26
4      38
5       6
6      15
7      28
8      21
9      15
10      9
11     17
12     17
13     18
14      8
15     26
16      3
17      6
18     21
19      4
20     11
21      7
22     23
23     11
24     15
25     16
26      8
27     24
28     12
29     43
       ..
784    41
785    34
786    26
787     9
788    35
789     4
790     3
791    22
792     4
793    24
794    13
795    27
796    11
797    21
798    17
799    13
800    21
801     3
802     2
803    14
804     0
805    12
806    22
807     2
808    19
809    10
810     6
811     5
812    11
813     1
Name: availableBikes, Length: 814, dtype: int64

In [27]:
bikes_bx = 0
for row in range(0,len(df)):
    if df['geometry'][row].within(boros.geometry[4])==True:
        bikes_bx = bikes_bx + df['availableBikes'][row]

In [28]:
bikes_Q = 0
for row in range(0,len(df)):
    if df['geometry'][row].within(boros.geometry[2])==True:
        bikes_Q = bikes_Q + df['availableBikes'][row]

In [29]:
bikes_bk = 0
for row in range(0,len(df)):
    if df['geometry'][row].within(boros.geometry[1])==True:
        bikes_bk = bikes_bk + df['availableBikes'][row]

In [30]:
bikes_si = 0
for row in range(0,len(df)):
    if df['geometry'][row].within(boros.geometry[0])==True:
        bikes_si = bikes_si + df['availableBikes'][row]

In [31]:
bikes_mn = 0
for row in range(0,len(df)):
    if df['geometry'][row].within(boros.geometry[3])==True:
        bikes_mn = bikes_mn + df['availableBikes'][row]

In [33]:
print ('available bikes in Manhattan', bikes_mn)
print ('available bikes in Staten Island', bikes_si)
print ('available bikes in Brooklyn', bikes_bk)
print ('available bikes in Queens', bikes_Q)
print ('available bikes in Bronx', bikes_bx)

available bikes in Manhattan 6500
available bikes in Staten Island 0
available bikes in Brooklyn 3652
available bikes in Queens 405
available bikes in Bronx 0
